In [37]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.5 MB/s eta 0:00:00


In [43]:
import pandas as pd
from pathlib import Path
from transformers import BartTokenizer, BartModel
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [4]:
DATA_DIR = Path("../data/steam")
PREPRO_WORK_DIR = DATA_DIR / "preprocessed"

In [5]:
games_df = pd.read_pickle(DATA_DIR / "steam_games.df.pkl")
games_df

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",NaN,NaN
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",4.24,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,NaN,NaN
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",1.39,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,NaN,NaN
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",1 user reviews,NaN


In [7]:
users_df = pd.read_pickle(PREPRO_WORK_DIR / "users_df.pkl")
users_df

,user_id,items_count,steam_id,user_url,items,reviews
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[{'funny': '', 'helpful': 'No ratings yet', 'i..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[{'funny': '', 'helpful': '15 of 20 people (75..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[{'funny': '', 'helpful': 'No ratings yet', 'i..."
3,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[{'funny': '', 'helpful': '2 of 2 people (100%..."
4,maplemage,629,76561198026584251,http://steamcommunity.com/id/maplemage,"[{'item_id': '240', 'item_name': 'Counter-Stri...","[{'funny': '3 people found this review funny',..."
...,...,...,...,...,...,...
26430,76561198306599751,7,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","[{'funny': '', 'helpful': '0 of 1 people (0%) ..."
26431,Ghoustik,4,76561198308105615,http://steamcommunity.com/id/Ghoustik,"[{'item_id': '302830', 'item_name': 'BLOCKADE ...","[{'funny': '', 'helpful': '0 of 1 people (0%) ..."
26432,76561198310819422,12,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'item_id': '268420', 'item_name': 'Aura King...","[{'funny': '1 person found this review funny',..."
26433,76561198312638244,36,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[{'funny': '', 'helpful': 'No ratings yet', 'i..."


In [8]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26435 entries, 0 to 26434
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      26435 non-null  object
 1   items_count  26435 non-null  int64 
 2   steam_id     26435 non-null  object
 3   user_url     26435 non-null  object
 4   items        26435 non-null  object
 5   reviews      26435 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.4+ MB


In [9]:
# drop_duplicates work around for list/array columns
users_df = users_df[~users_df.astype({"items": str, "reviews": str}).duplicated()]
users_df.shape

(25495, 6)

In [10]:
users_df['reviews'][0]

array([{'funny': '', 'helpful': 'No ratings yet', 'item_id': '1250', 'last_edited': '', 'posted': 'Posted November 5, 2011.', 'recommend': True, 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
       {'funny': '', 'helpful': 'No ratings yet', 'item_id': '22200', 'last_edited': '', 'posted': 'Posted July 15, 2011.', 'recommend': True, 'review': "It's unique and worth a playthrough."},
       {'funny': '', 'helpful': 'No ratings yet', 'item_id': '43110', 'last_edited': '', 'posted': 'Posted April 21, 2011.', 'recommend': True, 'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}],
      dtype=object)

In [11]:
def filter_users(df: pd.DataFrame):
    new_df = df.copy()
    new_df = new_df[new_df['reviews'].apply(lambda x : len(x) >=5)]
    new_df = new_df[new_df['items_count'] >= 5]
    return new_df

users_df = filter_users(users_df).reset_index(drop=True)
users_df

,user_id,items_count,steam_id,user_url,items,reviews
0,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[{'funny': '', 'helpful': 'No ratings yet', 'i..."
1,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[{'funny': '', 'helpful': '2 of 2 people (100%..."
2,DJKamBer,152,76561198026039330,http://steamcommunity.com/id/DJKamBer,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","[{'funny': '', 'helpful': '2 of 4 people (50%)..."
3,76561198070565427,75,76561198070565427,http://steamcommunity.com/profiles/76561198070...,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","[{'funny': '', 'helpful': 'No ratings yet', 'i..."
4,diego9031,200,76561198082027703,http://steamcommunity.com/id/diego9031,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","[{'funny': '', 'helpful': '0 of 1 people (0%) ..."
...,...,...,...,...,...,...
2956,TripleQMark,66,76561198202638474,http://steamcommunity.com/id/TripleQMark,"[{'item_id': '240', 'item_name': 'Counter-Stri...","[{'funny': '', 'helpful': '3 of 3 people (100%..."
2957,76561198208704549,45,76561198208704549,http://steamcommunity.com/profiles/76561198208...,"[{'item_id': '109600', 'item_name': 'Neverwint...","[{'funny': '1 person found this review funny',..."
2958,bestcustomurlevermade,21,76561198208817785,http://steamcommunity.com/id/bestcustomurlever...,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","[{'funny': '', 'helpful': '7 of 11 people (64%..."
2959,76561198236893796,17,76561198236893796,http://steamcommunity.com/profiles/76561198236...,"[{'item_id': '99900', 'item_name': 'Spiral Kni...","[{'funny': '', 'helpful': '4 of 6 people (67%)..."


In [12]:
users_df['items'][0][:5]

array([{'item_id': '1200', 'item_name': 'Red Orchestra: Ostfront 41-45', 'playtime_2weeks': 0, 'playtime_forever': 923},
       {'item_id': '1230', 'item_name': 'Mare Nostrum', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '1280', 'item_name': "Darkest Hour: Europe '44-'45", 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '1520', 'item_name': 'DEFCON', 'playtime_2weeks': 0, 'playtime_forever': 158},
       {'item_id': '220', 'item_name': 'Half-Life 2', 'playtime_2weeks': 0, 'playtime_forever': 1323}],
      dtype=object)

In [13]:
users_df['steam_id'].nunique()

2957

In [14]:
owned_games_df = users_df.explode("items", ignore_index=True)
owned_games_df

,user_id,items_count,steam_id,user_url,items,reviews
0,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"{'item_id': '1200', 'item_name': 'Red Orchestr...","[{'funny': '', 'helpful': 'No ratings yet', 'i..."
1,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"{'item_id': '1230', 'item_name': 'Mare Nostrum...","[{'funny': '', 'helpful': 'No ratings yet', 'i..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"{'item_id': '1280', 'item_name': 'Darkest Hour...","[{'funny': '', 'helpful': 'No ratings yet', 'i..."
3,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"{'item_id': '1520', 'item_name': 'DEFCON', 'pl...","[{'funny': '', 'helpful': 'No ratings yet', 'i..."
4,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"{'item_id': '220', 'item_name': 'Half-Life 2',...","[{'funny': '', 'helpful': 'No ratings yet', 'i..."
...,...,...,...,...,...,...
546967,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"{'item_id': '236110', 'item_name': 'Dungeon De...","[{'funny': '', 'helpful': '1 of 1 people (100%..."
546968,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"{'item_id': '417860', 'item_name': 'Emily is A...","[{'funny': '', 'helpful': '1 of 1 people (100%..."
546969,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"{'item_id': '427730', 'item_name': 'Who's Your...","[{'funny': '', 'helpful': '1 of 1 people (100%..."
546970,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"{'item_id': '449540', 'item_name': 'There's Po...","[{'funny': '', 'helpful': '1 of 1 people (100%..."


In [15]:
owned_games_df["items"][0]

{'item_id': '1200',
 'item_name': 'Red Orchestra: Ostfront 41-45',
 'playtime_2weeks': 0,
 'playtime_forever': 923}

In [16]:
games_temp_df = pd.DataFrame.from_records(owned_games_df["items"])
games_temp_df

,item_id,item_name,playtime_2weeks,playtime_forever
0,1200,Red Orchestra: Ostfront 41-45,0,923
1,1230,Mare Nostrum,0,0
2,1280,Darkest Hour: Europe '44-'45,0,0
3,1520,DEFCON,0,158
4,220,Half-Life 2,0,1323
...,...,...,...,...
546967,236110,Dungeon Defenders II,0,6
546968,417860,Emily is Away,0,76
546969,427730,Who's Your Daddy,0,21
546970,449540,There's Poop In My Soup,0,33


In [17]:
unique_game_ids_from_users = games_temp_df["item_id"].unique()
unique_game_ids_from_users, unique_game_ids_from_users.shape

(array(['1200', '1230', '1280', ..., '389900', '348280', '485270'],
       dtype=object),
 (9750,))

In [18]:
unique_game_ids_from_db = games_df["id"].unique()
unique_game_ids_from_db, unique_game_ids_from_db.shape

(array(['761140', '643980', '670290', ..., '610660', '658870', '681550'],
       dtype=object),
 (32133,))

In [19]:
games_ids_in_both_dfs = set(unique_game_ids_from_db).intersection(set(unique_game_ids_from_users))
len(games_ids_in_both_dfs)

8245

In [20]:
games_temp_df = games_temp_df[games_temp_df["item_id"].isin(games_ids_in_both_dfs)]
games_temp_df

,item_id,item_name,playtime_2weeks,playtime_forever
0,1200,Red Orchestra: Ostfront 41-45,0,923
1,1230,Mare Nostrum,0,0
2,1280,Darkest Hour: Europe '44-'45,0,0
3,1520,DEFCON,0,158
4,220,Half-Life 2,0,1323
...,...,...,...,...
546967,236110,Dungeon Defenders II,0,6
546968,417860,Emily is Away,0,76
546969,427730,Who's Your Daddy,0,21
546970,449540,There's Poop In My Soup,0,33


In [21]:
user_reviews_df = users_df.explode("reviews", ignore_index=True)
user_reviews_df

,user_id,items_count,steam_id,user_url,items,reviews
0,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","{'funny': '', 'helpful': 'No ratings yet', 'it..."
1,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","{'funny': '', 'helpful': 'No ratings yet', 'it..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","{'funny': '', 'helpful': 'No ratings yet', 'it..."
3,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","{'funny': '', 'helpful': 'No ratings yet', 'it..."
4,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","{'funny': '', 'helpful': 'No ratings yet', 'it..."
...,...,...,...,...,...,...
19999,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","{'funny': '', 'helpful': '1 of 1 people (100%)..."
20000,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","{'funny': '', 'helpful': 'No ratings yet', 'it..."
20001,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","{'funny': '', 'helpful': 'No ratings yet', 'it..."
20002,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","{'funny': '', 'helpful': 'No ratings yet', 'it..."


In [22]:
user_reviews_df["reviews"][0]

{'funny': '',
 'helpful': 'No ratings yet',
 'item_id': '248820',
 'last_edited': '',
 'posted': 'Posted February 3.',
 'recommend': True,
 'review': "A suitably punishing roguelike platformer.  Winning feels good.  Progressive unlocks mean a good slog ending in failure doesn't feel like a waste."}

In [23]:
reviews_temp_df = pd.DataFrame.from_records(user_reviews_df["reviews"])
reviews_temp_df

,funny,helpful,item_id,last_edited,posted,recommend,review
0,,No ratings yet,248820,,Posted February 3.,True,A suitably punishing roguelike platformer. Wi...
1,,No ratings yet,370360,"Last edited December 5, 2015.","Posted December 4, 2015.",True,"""Run for fun? What the hell kind of fun is that?"""
2,,No ratings yet,237930,,"Posted November 3, 2014.",True,"Elegant integration of gameplay, story, world ..."
3,,No ratings yet,263360,,"Posted October 15, 2014.",True,"Random drops and random quests, with stat poin..."
4,,No ratings yet,107200,,"Posted October 15, 2014.",True,Fun balance of tactics and strategy. Potentia...
...,...,...,...,...,...,...,...
19999,,1 of 1 people (100%) found this review helpful,224480,,Posted May 9.,True,Very fun game. very rich story line which flow...
20000,,No ratings yet,314160,,Posted March 30.,True,"FSX = BaeMust buy!!!Also, if you have a couple..."
20001,,No ratings yet,273110,,Posted March 26.,False,Well there is no other review I can give to th...
20002,,No ratings yet,264200,,Posted March 26.,True,One finger death punch is Bae. It's simplistic...


In [25]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartModel.from_pretrained('facebook/bart-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda


BartModel(
  (shared): Embedding(50265, 768, padding_idx=1)
  (encoder): BartEncoder(
    (embed_tokens): Embedding(50265, 768, padding_idx=1)
    (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
    (layers): ModuleList(
      (0-5): 6 x BartEncoderLayer(
        (self_attn): BartAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layernorm_embedding): La

In [32]:
def text_embedding(text, max_token_size=1024):

    inputs = tokenizer(text, return_tensors="pt").to(device)

    inputs['input_ids'] = inputs['input_ids'][:, :max_token_size]
    inputs['attention_mask'] = inputs['attention_mask'][:, :max_token_size]

    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state

    return last_hidden_states.detach().cpu().numpy()

reviews_temp_df["review_embedding"] = reviews_temp_df["review"].apply(text_embedding)

In [34]:
# 4gb reviews_temp_df.to_pickle("reviews_embedding_df.pkl")
reviews_temp_df

,funny,helpful,item_id,last_edited,posted,recommend,review,review_embedding
0,,No ratings yet,248820,,Posted February 3.,True,A suitably punishing roguelike platformer. Wi...,"[[[2.385041, 2.2305076, 0.7323009, 7.500683, 1..."
1,,No ratings yet,370360,"Last edited December 5, 2015.","Posted December 4, 2015.",True,"""Run for fun? What the hell kind of fun is that?""","[[[2.4143136, 2.1083455, 1.0787205, 8.29625, 1..."
2,,No ratings yet,237930,,"Posted November 3, 2014.",True,"Elegant integration of gameplay, story, world ...","[[[2.6618278, 2.8779314, 1.3105546, 8.57502, 1..."
3,,No ratings yet,263360,,"Posted October 15, 2014.",True,"Random drops and random quests, with stat poin...","[[[2.4476652, 2.802222, 1.3489541, 7.9748683, ..."
4,,No ratings yet,107200,,"Posted October 15, 2014.",True,Fun balance of tactics and strategy. Potentia...,"[[[2.8244557, 2.5118852, 1.2704822, 8.406587, ..."
...,...,...,...,...,...,...,...,...
19999,,1 of 1 people (100%) found this review helpful,224480,,Posted May 9.,True,Very fun game. very rich story line which flow...,"[[[2.9141235, 2.616892, 1.225038, 7.4030147, 1..."
20000,,No ratings yet,314160,,Posted March 30.,True,"FSX = BaeMust buy!!!Also, if you have a couple...","[[[2.9654503, 2.5194786, 1.0319756, 7.9356685,..."
20001,,No ratings yet,273110,,Posted March 26.,False,Well there is no other review I can give to th...,"[[[2.3869772, 2.4207978, 1.1481856, 7.899327, ..."
20002,,No ratings yet,264200,,Posted March 26.,True,One finger death punch is Bae. It's simplistic...,"[[[2.808123, 2.561251, 1.1975626, 7.672471, 1...."


In [38]:
!mkdir steam_pics
# to trwa 30h XD
!python3 scrape.py

  0% 3/8737 [00:39<31:37:06, 13.03s/it]
Exception ignored in: <module 'threading' from '/usr/lib/python3.10/threading.py'>
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1518, in _shutdown
    def _shutdown():
KeyboardInterrupt: 


In [76]:
model = models.resnet18(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])
model.to(device)
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [62]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = ImageFolder(root='steam_pics', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [86]:
class_to_idx = dataset.class_to_idx
idx_to_class = {idx: class_name for class_name, idx in class_to_idx.items()}

def features_from_images(model, dataloader, idx_to_class):
    id_to_features_dict = {}

    for images, labels in dataloader:
        with torch.no_grad():
            features = model(images.to(device)).flatten(start_dim=1).cpu().numpy()

        game_ids = [idx_to_class[label.item()] for label in labels]
        for game_id, feature in zip(game_ids, features):
            if game_id in id_to_features_dict:
                id_to_features_dict[game_id].append(feature)
            else:
                id_to_features_dict[game_id] = [feature]

    flattened_data = [(game_id, sublist) for game_id, sublists in id_to_features_dict.items() for sublist in sublists]
    df = pd.DataFrame(flattened_data, columns=['game_id', 'image_features'])
    return df


features_from_images(model, dataloader, idx_to_class)

,game_id,image_features
0,10,"[0.16411263, 0.17010887, 1.0292355, 1.5688009,..."
1,10,"[0.55013525, 0.72041965, 1.1867692, 1.9777325,..."
2,10,"[0.4055273, 0.2624749, 0.4552721, 1.8469104, 0..."
3,10,"[0.9047222, 0.27513358, 2.0699756, 0.74832636,..."
4,10,"[0.5541848, 0.717491, 1.2468506, 2.733252, 0.4..."
5,10,"[0.7402179, 3.8971438, 0.52749354, 1.9941632, ..."
6,1002,"[1.3545117, 0.50070673, 0.28013408, 1.5279653,..."
7,1002,"[1.7041358, 0.16183835, 1.9355913, 1.3031859, ..."
8,1002,"[1.4158622, 0.46312168, 0.38577574, 1.3773229,..."
9,1002,"[1.1487597, 0.19555171, 1.0626165, 0.9621998, ..."


In [ ]:
pivot_temp_df = pd.concat([owned_games_df, games_temp_df], axis=1)
pivot_temp_df

,user_id,items_count,steam_id,user_url,items,reviews,item_id,item_name,playtime_2weeks,playtime_forever
0,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"{'item_id': '1200', 'item_name': 'Red Orchestr...","[{'funny': '', 'helpful': 'No ratings yet', 'i...",1200,Red Orchestra: Ostfront 41-45,0.0,923.0
1,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"{'item_id': '1230', 'item_name': 'Mare Nostrum...","[{'funny': '', 'helpful': 'No ratings yet', 'i...",1230,Mare Nostrum,0.0,0.0
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"{'item_id': '1280', 'item_name': 'Darkest Hour...","[{'funny': '', 'helpful': 'No ratings yet', 'i...",1280,Darkest Hour: Europe '44-'45,0.0,0.0
3,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"{'item_id': '1520', 'item_name': 'DEFCON', 'pl...","[{'funny': '', 'helpful': 'No ratings yet', 'i...",1520,DEFCON,0.0,158.0
4,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"{'item_id': '220', 'item_name': 'Half-Life 2',...","[{'funny': '', 'helpful': 'No ratings yet', 'i...",220,Half-Life 2,0.0,1323.0
...,...,...,...,...,...,...,...,...,...,...
546967,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"{'item_id': '236110', 'item_name': 'Dungeon De...","[{'funny': '', 'helpful': '1 of 1 people (100%...",236110,Dungeon Defenders II,0.0,6.0
546968,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"{'item_id': '417860', 'item_name': 'Emily is A...","[{'funny': '', 'helpful': '1 of 1 people (100%...",417860,Emily is Away,0.0,76.0
546969,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"{'item_id': '427730', 'item_name': 'Who's Your...","[{'funny': '', 'helpful': '1 of 1 people (100%...",427730,Who's Your Daddy,0.0,21.0
546970,kimjongadam,57,76561198255497369,http://steamcommunity.com/id/kimjongadam,"{'item_id': '449540', 'item_name': 'There's Po...","[{'funny': '', 'helpful': '1 of 1 people (100%...",449540,There's Poop In My Soup,0.0,33.0
